In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
nn_model_id = 'retrained_sherlock'

In [3]:
import pandas as pd
import numpy as np

from sherlock.deploy.train_sherlock import train_sherlock
from sherlock.deploy.predict_sherlock import predict_sherlock
from datetime import datetime
from sklearn.metrics import f1_score, classification_report

### Load training and validation sets

In [4]:
start = datetime.now()
print(f'Started at {start}')

X_train = pd.read_parquet('train.parquet')
y_train = pd.read_parquet('../data/raw/train_labels.parquet').values.flatten()

y_train = np.array([x.lower() for x in y_train])

print(f'Load data (train) process took {datetime.now() - start} seconds.')

Started at 2022-02-02 08:00:37.888869
Load data (train) process took 0:00:03.905857 seconds.


In [5]:
print('Distinct types for columns in the Dataframe (should be all float32):')
print(set(X_train.dtypes))

Distinct types for columns in the Dataframe (should be all float32):
{dtype('float32')}


In [6]:
start = datetime.now()
print(f'Started at {start}')

X_validation = pd.read_parquet('validation.parquet')
y_validation = pd.read_parquet('../data/raw/val_labels.parquet').values.flatten()

y_validation = np.array([x.lower() for x in y_validation])

print(f'Load data (validation) process took {datetime.now() - start} seconds.')

Started at 2022-02-02 08:00:42.057951
Load data (validation) process took 0:00:02.116012 seconds.


### Re-train Sherlock

In [7]:
start = datetime.now()
print(f'Started at {start}')

train_sherlock(X_train, y_train, X_validation, y_validation, nn_id=nn_model_id);

print('Trained and saved new model.')
print(f'Finished at {datetime.now()}, took {datetime.now() - start} seconds')

Started at 2022-02-02 08:00:44.243257


W0202 08:00:44.731168 4681420288 deprecation.py:506] From /Users/lowecg/source/private-github/sherlock-project-1/venv/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0202 08:00:44.732186 4681420288 deprecation.py:506] From /Users/lowecg/source/private-github/sherlock-project-1/venv/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0202 08:00:44.735765 4681420288 deprecation.py:506] From /Users/lowecg/source/private-github/sherlock-project-1/venv/lib/python3.7/s

Successfully loaded and compiled model, now fitting model on data.
Train on 412059 samples, validate on 137353 samples


2022-02-02 08:00:48.532503: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-02-02 08:00:48.578998: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f8719f5eb30 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-02-02 08:00:48.579013: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


Epoch 1/100
412059/412059 [==============================] - 73s 177us/sample - loss: 1.6083 - categorical_accuracy: 0.6984 - val_loss: 1.0403 - val_categorical_accuracy: 0.8288
Epoch 2/100
412059/412059 [==============================] - 71s 173us/sample - loss: 0.9588 - categorical_accuracy: 0.8374 - val_loss: 0.9345 - val_categorical_accuracy: 0.8509
Epoch 3/100
412059/412059 [==============================] - 71s 172us/sample - loss: 0.8489 - categorical_accuracy: 0.8599 - val_loss: 0.8893 - val_categorical_accuracy: 0.8602
Epoch 4/100
412059/412059 [==============================] - 68s 164us/sample - loss: 0.7844 - categorical_accuracy: 0.8717 - val_loss: 0.8583 - val_categorical_accuracy: 0.8627
Epoch 5/100
412059/412059 [==============================] - 68s 165us/sample - loss: 0.7365 - categorical_accuracy: 0.8800 - val_loss: 0.8234 - val_categorical_accuracy: 0.8679
Epoch 6/100
412059/412059 [==============================] - 69s 166us/sample - loss: 0.6979 - categorical_acc

### Load test set

In [8]:
start = datetime.now()
print(f'Started at {start}')

X_test = pd.read_parquet('test.parquet')
y_test = pd.read_parquet('../data/raw/test_labels.parquet').values.flatten()

y_test = np.array([x.lower() for x in y_test])

print('Trained and saved new model.')
print(f'Finished at {datetime.now()}, took {datetime.now() - start} seconds')

Started at 2022-02-02 08:51:09.332895
Trained and saved new model.
Finished at 2022-02-02 08:51:11.300138, took 0:00:01.967257 seconds


### Make prediction

In [9]:
predicted_labels = predict_sherlock(X_test, nn_id=nn_model_id)

In [10]:
print(f'prediction count {len(predicted_labels)}, type = {type(predicted_labels)}')

size=len(y_test)

# Should be fully deterministic too.
f1_score(y_test[:size], predicted_labels[:size], average="weighted")

prediction count 137353, type = <class 'numpy.ndarray'>


0.8929132610917137

In [11]:
# Best results
# 0.8956975031011328, adam optimiser, took 0:59:19.320310 seconds (retrained_sherlock8)

# 0.8871638630500646, SGD Nesterov optimiser,  took 1:37:09.679227 seconds (retrained_sherlock9)

# LAST
# 0.8941992181577104
# 0.8661276466591463
# 0.8765360861865846
# 0.8777825108666634
# 0.8709458033528152
# 0.8526528126158138
# 0.836800124917518
# 0.8590214693450329
# 0.8583626471084315

In [12]:
classes = np.load(f"../sherlock/deploy/classes_{nn_model_id}.npy", allow_pickle=True)

report = classification_report(y_test, predicted_labels, output_dict=True)

class_scores = list(filter(lambda x: isinstance(x, tuple) and isinstance(x[1], dict) and 'f1-score' in x[1] and x[0] in classes, list(report.items())))

class_scores = sorted(class_scores, key=lambda item: item[1]['f1-score'], reverse=True)

### Top 5 Types

In [13]:
print(f"\t\tf1-score\tprecision\trecall\t\tsupport")

for key, value in class_scores[0:5]:
    if len(key) >= 8:
        tabs = '\t' * 1
    else:
        tabs = '\t' * 2

    print(f"{key}{tabs}{value['f1-score']:.3f}\t\t{value['precision']:.3f}\t\t{value['recall']:.3f}\t\t{value['support']}")

		f1-score	precision	recall		support
grades		0.992		0.990		0.994		1765
isbn		0.989		0.987		0.991		1430
jockey		0.987		0.984		0.989		2819
industry	0.982		0.978		0.986		2958
currency	0.976		0.987		0.965		405


### Bottom 5 Types

In [14]:
print(f"\t\tf1-score\tprecision\trecall\t\tsupport")

for key, value in class_scores[len(class_scores)-5:len(class_scores)]:
    if len(key) >= 8:
        tabs = '\t' * 1
    else:
        tabs = '\t' * 2

    print(f"{key}{tabs}{value['f1-score']:.3f}\t\t{value['precision']:.3f}\t\t{value['recall']:.3f}\t\t{value['support']}")

		f1-score	precision	recall		support
rank		0.705		0.681		0.731		2983
person		0.650		0.679		0.623		579
sales		0.550		0.722		0.444		322
director	0.549		0.634		0.484		225
ranking		0.461		0.753		0.333		439


### All Scores

In [15]:
print(classification_report(y_test, predicted_labels, digits=3))

                precision    recall  f1-score   support

       address      0.921     0.943     0.932      3003
     affiliate      0.965     0.804     0.877       204
   affiliation      0.978     0.951     0.964      1768
           age      0.852     0.956     0.901      3033
         album      0.892     0.880     0.886      3035
          area      0.873     0.825     0.848      1987
        artist      0.802     0.878     0.839      3043
    birth date      0.975     0.975     0.975       479
   birth place      0.977     0.904     0.939       418
         brand      0.800     0.704     0.749       574
      capacity      0.793     0.721     0.755       362
      category      0.925     0.890     0.907      3087
          city      0.844     0.898     0.870      2966
         class      0.896     0.920     0.908      2971
classification      0.925     0.864     0.893       587
          club      0.976     0.948     0.962      2977
          code      0.930     0.906     0.918  

## Review errors

In [20]:
from collections import Counter

size = len(y_test)
mismatches = list()

for idx, k1 in enumerate(y_test[:size]):
    k2 = predicted_labels[idx]

    if k1 != k2:
        mismatches.append(k1)
        
        # zoom in to specific errors. Use the index in the next step
        if k1 in ('address'):
            print(f'[{idx}] expected "{k1}" but predicted "{k2}"')
        
f1 = f1_score(y_test[:size], predicted_labels[:size], average="weighted")
print(f'Total mismatches: {len(mismatches)} (F1 score: {f1})')

data = Counter(mismatches)
data.most_common()   # Returns all unique items and their counts

[1001] expected "address" but predicted "location"
[1116] expected "address" but predicted "name"
[2588] expected "address" but predicted "location"
[2616] expected "address" but predicted "location"
[3398] expected "address" but predicted "city"
[3530] expected "address" but predicted "location"
[4380] expected "address" but predicted "county"
[5112] expected "address" but predicted "location"
[5647] expected "address" but predicted "team name"
[5780] expected "address" but predicted "sales"
[8797] expected "address" but predicted "area"
[9116] expected "address" but predicted "credit"
[9354] expected "address" but predicted "location"
[9574] expected "address" but predicted "location"
[9806] expected "address" but predicted "city"
[10035] expected "address" but predicted "creator"
[10067] expected "address" but predicted "rank"
[10665] expected "address" but predicted "area"
[11055] expected "address" but predicted "city"
[11993] expected "address" but predicted "location"
[12639] ex

Total mismatches: 14657 (F1 score: 0.8929132610917137)


[('rank', 802),
 ('name', 786),
 ('location', 551),
 ('region', 487),
 ('position', 444),
 ('description', 433),
 ('notes', 400),
 ('team', 395),
 ('artist', 370),
 ('album', 363),
 ('type', 360),
 ('area', 348),
 ('category', 340),
 ('company', 332),
 ('product', 331),
 ('day', 302),
 ('city', 302),
 ('ranking', 293),
 ('gender', 285),
 ('code', 279),
 ('team name', 248),
 ('class', 237),
 ('country', 222),
 ('person', 218),
 ('year', 213),
 ('owner', 212),
 ('weight', 202),
 ('result', 189),
 ('status', 189),
 ('sales', 179),
 ('manufacturer', 178),
 ('sex', 176),
 ('duration', 171),
 ('address', 171),
 ('credit', 170),
 ('brand', 170),
 ('service', 170),
 ('club', 155),
 ('component', 151),
 ('range', 149),
 ('state', 148),
 ('origin', 143),
 ('plays', 137),
 ('order', 137),
 ('format', 134),
 ('age', 132),
 ('county', 126),
 ('director', 116),
 ('capacity', 101),
 ('nationality', 92),
 ('affiliation', 86),
 ('command', 85),
 ('symbol', 82),
 ('classification', 80),
 ('family', 78),

In [17]:
test_samples = pd.read_parquet('../data/raw/test_values.parquet')

In [21]:
from ast import literal_eval

idx = 1001
original = test_samples.iloc[idx]
converted = original.apply(literal_eval).to_list()

print(f'Predicted "{predicted_labels[idx]}", actual label "{y_test[idx]}". Actual values:\n{converted}')

Predicted "location", actual label "address". Actual values:
[['Cabot House', 'Cabot House', '5 Hill Rd.', '5 Hill Rd.', '9 Cabot Rd.', '9 Cabot Rd.', 'Cabot House', '22 Bank Rd.', '22 Bank Rd.', 'Cabot House', '31 Bank Rd.', '31 Bank Rd.', 'Bairds Hotel', '11 Cabot Rd.', '11 Cabot Rd.', '10 Hill Rd.', '10 Hill Rd.', '10 Hill Rd.', '10 Hill Rd.', '7A Church Rd.', '1 Cabot Rd.', '1 Cabot Rd.', '1 Cabot Rd.', '1 Cabot Rd.', '2 Coronation St.', '2 Coronation St.', '7A Church Rd.', '12 Hill Rd.', '12 Hill Rd.', '12 Hill Rd.', 'Cabot House', '19 Bank Rd.', '19 Bank Rd.', '19 Bank Rd.', '19 Bank Rd.', '19 Bank Rd.', '7A Church Rd.', '18 Mill Rd.', '17 Hill Rd.', '17 Hill Rd.', 'Cabot House', 'Cabot House', '25 Bank Rd.', '10 Coronation St.', '6 Cabot Rd.', '6 Cabot Rd.', '8 Hill Rd.', '8 Hill Rd.', '4 Mill Rd.', '4 Mill Rd.', '12 Sulva Rd.', '4 Haig Rd.', '13 Botwood Rd.', '13 Botwood Rd.', '8 Botwood Rd.', '8 Botwood Rd.', '16 Botwood Rd.', '16 Botwood Rd.', '16 Botwood Rd.', '16 Botwood Rd